# Chapter 12. Serial Correlation and Heteroskedasticity in Time Series Regressions

In this chapter, we discuss the critical problem of serial correlation in the error terms of a multiple regression model. Static and finite distributed lag models often have serially correlated errors even if there is no underlying misspecification of the model. Therefore, it is important to know the consequences and remedies for serial correlation for these useful classes of models.

In Section 12-1, we present the properties of OLS when the errors contain serial correlation. In Section 12-2, we demonstrate how to test for serial correlation. We cover tests that apply to mod- els with strictly exogenous regressors and tests that are asymptotically valid with general regressors, including lagged dependent variables. Section 12-3 explains how to correct for serial correlation under the assumption of strictly exogenous explanatory variables, while Section 12-4 shows how using differenced data often eliminates serial correlation in the errors. Section 12-5 covers more recent advances on how to adjust the usual OLS standard errors and test statistics in the presence of very general serial correlation.

In Chapter 8, we discussed testing and correcting for heteroskedasticity in cross-sectional appli- cations. In Section 12-6, we show how the methods used in the cross-sectional case can be extended to the time series case. The mechanics are essentially the same, but there are a few subtleties associ- ated with the temporal correlation in time series observations that must be addressed. In addition, we briefly touch on the consequences of dynamic forms of heteroskedasticity.

## 12-1. Properties of OLS with Serially Correlated Errors

### 12-1a Unbiasedness and Consistency

In Chapter 10, we proved unbiasedness of the OLS estimator under the first three Gauss-Markov assumptions for time series regressions (TS.1 through TS.3). In particular, Theorem 10.1 assumed nothing about serial correlation in the errors. It follows that, as long as the explanatory variables are strictly exogenous, the $\hat \beta_j$ are unbiased, regardless of the degree of serial correlation in the errors. This is analogous to the observation that heteroskedasticity in the errors does not cause bias in the $\hat \beta_j$ .

In Chapter 11, we relaxed the strict exogeneity assumption to $E(u_t|x_t)$ and showed that, when the data are weakly dependent, the $\hat \beta_j$ are still consistent (although not necessarily unbiased). This result did not hinge on any assumption about serial correlation in the errors.

### 12-1b Efficiency and Inference

Because the Gauss-Markov Theorem (Theorem 10.4) requires both homoskedasticity and serially uncorrelated errors, OLS is no longer BLUE in the presence of serial correlation. Even more impor- tantly, the usual OLS standard errors and test statistics are not valid, even asymptotically.

## 12-2 Testing for Serial Correlation

In this section, we discuss several methods of testing for serial correlation in the error terms in the multiple linear regression model

\begin{equation}
y_t=\beta_0+\beta_1 x_{t1}+\ldots+\beta_k x_{tk}+ u_t \tag{12.2}
\end{equation}

We first consider the case when the regressors are strictly exogenous. Recall that this requires the error, $u_t$ , to be uncorrelated with the regressors in all time periods (see Section 10-3), so, among other things, it rules out models with lagged dependent variables.

### 12-2a A t Test for AR(1) Serial Correlation with Strictly Exogenous Regressors

Suppose we are worried that the error terms $u_1,u_2,\ldots$ in model (12.2) are serially correlated. As described in Wooldridge 2016 an intuitive testing approach is based on the fitted residuals $\hat \u_t=y_t-\hat \beta_0-\hat \beta_1 x_{t1}-\ldots-\hat \beta_1 x_{t1}$. To test for AR(1) with serial correlation under strict exogeneity, we regress $\hat u_t$ on their lagged values $\hat u_{t-1}$. Then we perform the usual t test on the coefficients $\hat u_{t-1}$.

### Wooldridge Example 12.1. Testing for AR(1) Serial Correlation in the Phillips Curve

In Chapter 10, we estimated a static Phillips curve that explained the inflation-unemployment tradeoff in the United States (see Example 10.1). In Chapter 11, we studied a particular expectations aug- mented Phillips curve, where we assumed adaptive expectations (see Example 11.5). We now test the error term in each equation for serial correlation. Since the expectations augmented curve uses $\Delta inf_t=inf_t-inf_{t-1}$ as the dependent variable, we have one fewer observation.

In [1]:
library(foreign);library(dynlm);library(lmtest)

Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



In [6]:
phillips <- read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/phillips.dta?raw=true")

# Define Yearly time series beginning in 1948
tsdata <- ts(phillips, start=1948)

# Estimation of static Phillips curve:
reg.s <- dynlm( inf ~ unem, data=tsdata, end=1996)

# residuals and AR(1) test:
residual.s <- resid(reg.s)
coeftest( dynlm(residual.s ~ L(residual.s)) )


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -0.11340    0.35940 -0.3155    0.7538    
L(residual.s)  0.57297    0.11613  4.9337 1.098e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


For the static Phillips curve, the regression in (12.14) yields $\hat \rho=.573,t=4.93$, and p-value=.000 (with 48 observations through 1996). This is very strong evidence of positive, first order serial correlation. One consequence of this is that the standard errors and t statistics from Chapter 10 are not valid.

By contrast, the test for AR(1) serial correlation in the expectations augmented curve, refer to the following code, gives $\hat \rho=-.355,t=-.287$ and p-value=.775 (with 47 observations): there is no evidence of AR(1) serial correlation in the expectations augmented Phillips curve.

In [7]:
# Same with expectations-augmented Phillips curve:
reg.ea <- dynlm( d(inf) ~ unem, data=tsdata, end=1996)
residual.ea <- resid(reg.ea)
coeftest( dynlm(residual.ea ~ L(residual.ea)) )



t test of coefficients:

                Estimate Std. Error t value Pr(>|t|)
(Intercept)     0.194166   0.300384  0.6464   0.5213
L(residual.ea) -0.035593   0.123891 -0.2873   0.7752


Although the test from (12.14) is derived from the AR(1) model, the test can detect other kinds of serial correlation. Remember, $\hat \rho$ is a consistent estimator of the correlation between $u_t$ and $u_{t-1}$ . Any serial correlation that causes adjacent errors to be correlated can be picked up by this test. On the other hand, it does not detect serial correlation where adjacent errors are uncorrelated,

### 12-2d Testing for Higher Order Serial Correlation

### Wooldridge Example 12.3. Testing for AR(3) Serial Correlation

In the event study of the barium chloride industry (see Example 10.5), we used monthly data, so we may wish to test for higher orders of serial correlation. For illustration purposes, we test for AR(3) serial correlation in the errors underlying equation (10.22).

In [8]:
library(foreign);library(dynlm);library(car);library(lmtest)

In [10]:
barium<-read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/barium.dta?raw=true")

tsdata <- ts(barium, start=c(1978,2), frequency=12)

reg <- dynlm(log(chnimp)~log(chempi)+log(gas)+log(rtwex)+
                                  befile6+affile6+afdec6, data=tsdata )

# Automatic test:
bgtest(reg, order=3, type="F")


	Breusch-Godfrey test for serial correlation of order up to 3

data:  reg
LM test = 5.1247, df1 = 3, df2 = 121, p-value = 0.002264


Using the R command bgtest we find that the F-statistic for joint significance of $\hat \u_{t-1},\hat \u_{t-3},\hat \u_{t-3}$ is F=5.12 with a p-value of .00226 thus providing strong evidence of AR(3) serial correlation.

## 12-3 Correcting for Serial Correlation with Strictly Exogenous Regressors

If we detect serial correlation after applying one of the tests in Section 12-2, we have to do something about it. If our goal is to estimate a model with complete dynamics, we need to respecify the model. In applications where our goal is not to estimate a fully dynamic model, we need to find a way to carry out statistical inference: as we saw in Section 12-1, the usual OLS test statistics are no longer valid. In this section, we begin with the important case of AR(1) serial correlation. The traditional approach to this problem assumes fixed regressors. What are actually needed are strictly exogenous regressors. Therefore, at a minimum, we should not use these corrections when the explanatory variables include lagged dependent variables.

### 12-3b Feasible GLS Estimation with AR(1) Errors

Refer to Wooldridge 2016 for detail information on FLGS methods.

### Wooldridge Example 12.4 Prais-Winsten Estimation in the Event Study

In [ ]:
Again using the data in BARIUM, we estimate the equation in Example 10.5 using iterated Prais- Winsten estimation. For comparison, we also present the OLS results

In [21]:
install.packages('orcutt')
install.packages('stargazer')
library(foreign);library(dynlm);library(car);library(orcutt);library(stargazer)

Installing package into '/home/nbuser/R'
(as 'lib' is unspecified)
Installing package into '/home/nbuser/R'
(as 'lib' is unspecified)

Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.1. https://CRAN.R-project.org/package=stargazer 



In [25]:
barium<-read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/barium.dta?raw=true")


tsdata <- ts(barium, start=c(1978,2), frequency=12)

# OLS estimation
olsres <- dynlm(log(chnimp)~log(chempi)+log(gas)+log(rtwex)+
      befile6+affile6+afdec6, data=tsdata)

# Cochrane-Orcutt estimation
praiswinstein <-cochrane.orcutt(olsres)

stargazer(olsres,praiswinstein,type="text")


                          Dependent variable:      
                    -------------------------------
                              log(chnimp)          
                             (1)             (2)   
---------------------------------------------------
log(chempi)                3.117***        2.947***
                           (0.479)         (0.646) 
                                                   
log(gas)                    0.196           1.055  
                           (0.907)         (0.991) 
                                                   
log(rtwex)                 0.983**         1.137** 
                           (0.400)         (0.514) 
                                                   
befile6                     0.060           -0.016 
                           (0.261)         (0.321) 
                                                   
affile6                     -0.032          -0.033 
                           (0.264)         (0.323) 
           

The coefficients that are statistically significant in the Prais-Winsten estimation do not differ by much from the OLS estimates [in particular, the coefficients on log( chempi ), log( rtwex ), and afdec6 ]. It is not surprising for statistically insignificant coefficients to change, perhaps markedly, across different estimation methods.

Notice how the standard errors in the second column are uniformly higher than the standard errors in first column. This is common. The Prais-Winsten standard errors account for serial correlation; the OLS standard errors do not. As we saw in Section 12-1, the OLS standard errors usually understate the actual sampling variation in the OLS estimates and should not be relied upon when significant serial correlation is present. Therefore, the effect on Chinese imports after the International Trade Commission's decision is now less statistically significant than we thought.

## 12.4 Differencing and Serial Correlation

In Chapter 11, we presented differencing as a transformation for making an integrated process weakly dependent. There is another way to see the merits of differencing when dealing with highly persistent data. Suppose that we start with the simple regression model:

\begin{equation}
y_t=\beta_0+\beta_1 x_{t}+ u_t,t=1,2,\ldots \tag{12.37}
\end{equation}

where u t follows the AR(1) process in (12.26). As we mentioned in Section 11-3, and as we will discuss more fully in Chapter 18, the usual OLS inference procedures can be very misleading when the variables $y_t$ and $x_t$ are integrated of order one, or I(1). In the extreme case where the errors ${u_t}$ in (12.37) follow a random walk, the equation makes no sense because, among other things, the variance of $u_t$ grows with t . It is more logical to difference the equation:

\begin{equation}
\Delta y_t=\beta_1 \Delta x_{t}+ \Delta u_t,t=2,\ldots,n \tag{12.38}
\end{equation}

If $u_t$ follows a random walk, then $e_t=\Delta u_t$ has zero mean and a constant variance and is serially uncorrelated. Thus, assuming that $e_t$ and $\Delta x_t$ are uncorrelated, we can estimate (12.38) by OLS, where we lose the first observation.

Even if $u_t$ does not follow a random walk, but $\rho$ is positive and large, first differencing is often a good idea: it will eliminate most of the serial correlation. As we explained in Chapter 11, the decision of whether or not to difference is a tough one. But this discussion points out another benefit of differencing, which is that it removes serial correlation. We will come back to this issue in Chapter 18.

## 12-5 Serial Correlation-Robust Inference after OLS

Unbiasedness and consistency of OLS are not affected by heteroskedasticity or serial correlation, but the standard erros are. In recent years, it has become more popular to estimate models by OLS but to correct the standard errors for fairly arbitrary forms of serial correlation (and heteroskedasticity). Even though we know OLS will be inefficient, there are some good reasons for taking this approach. First, the explanatory variables may not be strictly exogenous. In this case, FGLS is not even consistent, let alone efficient. Second, in most applications of FGLS, the errors are assumed to follow an AR(1) model. It may be better to compute standard errors for the OLS estimates that are robust to more general forms of serial correlation.

Similar to heteroskedasticity-robust standard errors discussed in chapter 8, there are mechanisms valid in the case of time series, refer to Wooldridge 2016 for more information.

### Wooldridge Example 12.7. The Puerto Rican Minimum Wage

We obtain an SC-robust standard error for the minimum wage effect in the Puerto Rican employment equation. In Example 12.2, we found pretty strong evidence of AR(1) serial correlation. As in that example, we use as additional controls log(usgnp), log(prgnp), and a linear time trend.

In [7]:
library(foreign);library(dynlm);library(lmtest);library(sandwich)


Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.1. https://CRAN.R-project.org/package=stargazer 



In [10]:
minwg<-read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/prminwge.dta?raw=true")

tsdata <- ts(minwg, start=1950)

# OLS regression
reg<-dynlm(log(prepop)~log(mincov)+log(prgnp)+log(usgnp)+trend(tsdata), 
                                                          data=tsdata )
# results with usual SE
coeftest(reg)
# results with HAC SE
coeftest(reg, vcovHAC)



t test of coefficients:

                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -6.6634416  1.2578286 -5.2976 7.667e-06 ***
log(mincov)   -0.2122612  0.0401523 -5.2864 7.924e-06 ***
log(prgnp)     0.2852380  0.0804921  3.5437  0.001203 ** 
log(usgnp)     0.4860482  0.2219825  2.1896  0.035731 *  
trend(tsdata) -0.0266633  0.0046267 -5.7629 1.940e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1



t test of coefficients:

                Estimate Std. Error t value  Pr(>|t|)    
(Intercept)   -6.6634416  1.6856887 -3.9529 0.0003845 ***
log(mincov)   -0.2122612  0.0460683 -4.6075 5.835e-05 ***
log(prgnp)     0.2852380  0.1034900  2.7562 0.0094497 ** 
log(usgnp)     0.4860482  0.3108939  1.5634 0.1275013    
trend(tsdata) -0.0266633  0.0054301 -4.9103 2.402e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


The OLS estimate of the elasticity of the employment rate with respect to the minimum wage is $\hat \beta_1=-.2122$, and the usual OLS standard error is $se(\hat \beta_1=.0401)$. The SC-robust standard error $se(\hat \beta_1)=.046$. Interestingly, the robust standard error is only slightly greater than the usual OLS standard error. The robust t statistic is about -4.60, and so the estimated elasticity is still statistically significant.

## 12.6. Heteroskedasticity in Time Series Regressions

We discussed testing and correcting for heteroskedasticity for cross-sectional applications in Chapter 8. Heteroskedasticity can also occur in time series regression models, and the presence of heteroskedasticity, while not causing bias or inconsistency in the $\hat beta_j$ , does invalidate the usual standard errors, t statistics, and F statistics. This is just as in the cross-sectional case.

In time series regression applications, heteroskedasticity often receives little, if any, attention: the problem of serially correlated errors is usually more pressing. Nevertheless, it is useful to briefly cover some of the issues that arise in applying tests and corrections for heteroskedasticity in time series regressions.

### 12-6a Heteroskedasticity-Robust Statistics

In studying heteroskedasticity for cross-sectional regressions, we noted how it has no bearing on the unbiasedness or consistency of the OLS estimators. Exactly the same conclusions hold in the time series case, as we can see by reviewing the assumptions needed for unbiasedness (Theorem 10.1) and consistency (Theorem 11.1).

In Section 8-2, we discussed how the usual OLS standard errors, t statistics, and F statistics can be adjusted to allow for the presence of heteroskedasticity of unknown form. These same adjustments work for time series regressions under Assumptions TS.1', TS.2', TS.3', and TS.5'. Thus, provided the only assumption violated is the homoskedasticity assumption, valid inference is easily obtained in most econometric packages.

### 12-6b Testing for Heteroskedasticity

Sometimes, we wish to test for heteroskedasticity in time series regressions, especially if we are concerned about the performance of heteroskedasticity-robust statistics in relatively small sample sizes. The tests we covered in Chapter 8 can be applied directly, but with a few caveats. First, the errors $u_t$ should not be serially correlated; any serial correlation will generally invalidate a test for heteroskedasticity. Thus, it makes sense to test for serial correlation first, using a heteroskedasticity-robust test if heteroskedasticity is suspected. Then, after something has been done to correct for serial correlation, we can test for heteroskedasticity.

### Wooldridge Example 12.8. Heteroskedasticity and the Efficient Markets Hypothesis

TO DO

### 12-6c Autoregressive Conditional Heteroskedasticity (ARCH)

In recent years, economists have become interested in dynamic forms of heteroskedasticity. Of course, if $x_t$ contains a lagged dependent variable, then heteroskedasticity as in (12.46) is dynamic. But dynamic forms of heteroskedasticity can appear even in models with no dynamics in the regression equation.

### Wooldridge Example 12.9 ARCH in Stock Returns

Heteroskedasticity is quite common in finantial data (e.g. weekly stock returns). This heteroskedasticity is actually better characterized by the ARCH models (refer to Wooldridge 2016). For the case of NYSE data if we compute a simple AR(1) model, square the residuals, and regress them on the lagged squared residual, we obtain:

In [11]:
library(foreign);library(dynlm);library(lmtest)
nyse <- read.dta("https://github.com/thousandoaks/Wooldridge/blob/master/nyse.dta?raw=true")

tsdata <- ts(nyse)

# Linear regression of model:
reg <- dynlm(return ~ L(return), data=tsdata) 

# squared residual
residual.sq <- resid(reg)^2

# Model for squared residual:
ARCHreg <- dynlm(residual.sq ~ L(residual.sq)) 
coeftest(ARCHreg)



t test of coefficients:

               Estimate Std. Error t value  Pr(>|t|)    
(Intercept)    2.947434   0.440234  6.6951 4.485e-11 ***
L(residual.sq) 0.337062   0.035947  9.3767 < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1


The t-statistic on $\hat u^2_{t-1}$ is 9.37, indicating strong ARCH. Larger errors at time t-1 imply larger variance in stock returns today.

### 12-6d Heteroskedasticity and Serial Correlation in Regression Models

Nothing rules out the possibility of both heteroskedasticity and serial correlation being present in a regression model. If we are unsure, we can always use OLS and compute fully robust standard errors, as described in Section 12-5.

Much of the time serial correlation is viewed as the most important problem, because it usually has a larger impact on standard errors and the efficiency of estimators than does heteroskedasticity. As described in Wooldridge 2016 (Chapter 12), obtaining tests for serial correlation that are robust to arbitrary heteroskedasticity is fairly straightforward. If we detect serial correlation using such a test, we can employ the Cochrane-Orcutt (or Prais-Winsten) transformation and, in the transformed equation, use heteroskedasticity-robust standard errors and test statistics. Or, we can even test for heteroskedasticity in using the Breusch-Pagan or White tests.

## Summary

We have covered the important problem of serial correlation in the errors of multiple regression models. Positive correlation between adjacent errors is common, especially in static and finite distributed lag models. This causes the usual OLS standard errors and statistics to be misleading (although the b ^ j can still be un- biased, or at least consistent). Typically, the OLS standard errors underestimate the true uncertainty in the parameter estimates.

The most popular model of serial correlation is the AR(1) model. Using this as the starting point, it is easy to test for the presence of AR(1) serial correlation using the OLS residuals. An asymptotically valid t statistic is obtained by regressing the OLS residuals on the lagged residuals, assuming the regressors are strictly exogenous and a homoskedasticity assumption holds. Making the test robust to heteroskedasticity is simple.

For models with a lagged dependent variable or other nonstrictly exogenous regressors, the standard t test on $\hat u_{t-1}$ is still valid, provided all independent variables are included as regressors along with $\hat u_{t-1}$ . We can use an F or an LM statistic to test for higher order serial correlation.

In models with strictly exogenous regressors, we can use a feasible GLS procedure -Cochrane-Orcutt or Prais-Winsten- to correct for AR(1) serial correlation. This gives estimates that are different from the OLS estimates: the FGLS estimates are obtained from OLS on quasi-differenced variables. All of the usual test statistics from the transformed equation are asymptotically valid. Almost all regression packages have built-in features for estimating models with AR(1) errors.

Another way to deal with serial correlation, especially when the strict exogeneity assumption might fail, is to use OLS but to compute serial correlation-robust standard errors (that are also robust to heteroskedasticity). Many regression packages follow a method suggested by Newey and West (1987); it is also possible to use standard regression packages to obtain one standard error at a time.

Finally, we discussed some special features of heteroskedasticity in time series models. As in the cross-sectional case, the most important kind of heteroskedasticity is that which depends on the explanatory variables; this is what determines whether the usual OLS statistics are valid. The Breusch-Pagan and White tests covered in Chapter 8 can be applied directly, with the caveat that the errors should not be serially correlated. In recent years, economists -especially those who study the financial markets- have become interested in dynamic forms of heteroskedasticity. The ARCH model is the leading example.